In [ ]:
import influxdb_client
import pandas as pd
from influxdb_client.client.write_api import SYNCHRONOUS
import networkx as nx


# InfluxDB instance details

bucket = "schempp"
org = "UMA"
token = "TOKEN_ID"
# Store the URL of your InfluxDB instance
url="http://IP_ADDRESS:PORT"

client = influxdb_client.InfluxDBClient(url=url,token=token,org=org)

#Query delay
query_api = client.query_api()


In [ ]:
#Downlink query
dl_query = 'from(bucket: "schempp")\
|> range(start: 2023-05-01T05:40:00Z, stop: 2023-05-01T12:10:00Z)\
|> filter(fn: (r) => r["parameter"] == "dlMaxRetxThreshold" or r["parameter"] == "dlPollByte" or r["parameter"] == "dlPollPDU" or r["parameter"] == "dlTPollRetr" or r["parameter"] == "dlTProhib" or r["parameter"] == "dlTReassembly")\
|> sort()\
|> yield(name: "sort")'

df_dl_results = query_api.query_data_frame(query=dl_query)

#Uplink query
ul_query = 'from(bucket: "schempp")\
|> range(start: 2023-05-01T06:10:00Z, stop: 2023-05-01T07:10:00Z)\
|> filter(fn: (r) => r["parameter"] == "ulMaxRetxThreshold" or r["parameter"] == "ulPollByte" or r["parameter"] == "ulPollPDU" or r["parameter"] == "ulTPollRetr" or r["parameter"] == "ulTProhib" or r["parameter"] == "ulTReassembly")\
|> sort()\
|> yield(name: "sort")'
ul_results = query_api.query_data_frame(query=ul_query)

#Delay query
query_api = client.query_api()
delay_query = 'from(bucket: "schempp")\
|> range(start: 2023-05-01T06:10:00Z, stop: 2023-05-01T12:10:00Z)\
|> filter(fn: (r) => r["_measurement"] == "kpis")\
|> filter(fn: (r) => r["_field"] == "delay")\
|> aggregateWindow(every: 1s, fn: mean, createEmpty: false)\
|> yield(name: "mean")'

df_delay_result = query_api.query_data_frame(query=delay_query)

#Jitter query
query_api = client.query_api()
jitter_query = 'from(bucket: "schempp")\
|> range(start: 2023-05-01T06:10:00Z, stop: 2023-05-01T12:10:00Z)\
|> filter(fn: (r) => r["_measurement"] == "kpis")\
|> filter(fn: (r) => r["_field"] == "jitter")\
|> aggregateWindow(every: 1s, fn: mean, createEmpty: false)\
|> yield(name: "mean")'

df_jitter_result = query_api.query_data_frame(query=jitter_query)

In [ ]:
def getConfig(config_df_a, time):
    for i in range(len(config_df_a)):
        #Previous configuration
        if (i < (len(config_df_a)-1)):
            time_for = config_df_a.iloc[i]['time'].tz_localize(None)
            time_for_next = config_df_a.iloc[i+1]['time'].tz_localize(None)
            if ((time_for <= time.tz_localize(None)) and (time_for_next > time.tz_localize(None))):
                return config_df_a.iloc[i].T
        #Current configuration    
        elif (i == len(config_df_a)-1):
            time_for = config_df_a.iloc[i]['time'].tz_localize(None)
            if (time_for <= time.tz_localize(None)):
                return config_df_a.iloc[i].T
            #Unexpected behavior
            else:
                print("[ERROR] There is no configuration learned at the time & date provided:")
                print(time.tz_localize(None))
                return None


def addConfig(configToAppend):
    global total_configs_df
    if (getConfigID(configToAppend) == -1):
        total_configs_df = total_configs_df.append(configToAppend, ignore_index=True)
    else:
        print("Config already exists")


def getConfigID(configToCheck):
    global total_configs_df
    for i in range(len(total_configs_df)):
        if (total_configs_df.iloc[i]['dlMaxRetxThreshold'] == configToCheck['dlMaxRetxThreshold'] and
            total_configs_df.iloc[i]['dlPollByte'] == configToCheck['dlPollByte'] and
            total_configs_df.iloc[i]['dlPollPDU'] == configToCheck['dlPollPDU'] and
            total_configs_df.iloc[i]['dlTPollRetr'] == configToCheck['dlTPollRetr'] and
            total_configs_df.iloc[i]['dlTProhib'] == configToCheck['dlTProhib'] and
            total_configs_df.iloc[i]['dlTReassembly'] == configToCheck['dlTReassembly']):
            return i
    return -1

mask_dlMaxRetxThreshold=df_dl_results.loc[:,'parameter'] == "dlMaxRetxThreshold"
mask_dlPollByte=df_dl_results.loc[:,'parameter'] == "dlPollByte"
mask_dlPollPDU=df_dl_results.loc[:,'parameter'] == "dlPollPDU"
mask_dlTPollRetr=df_dl_results.loc[:,'parameter'] == "dlTPollRetr"
mask_dlTProhib=df_dl_results.loc[:,'parameter'] == "dlTProhib"
mask_dlTReassembly=df_dl_results.loc[:,'parameter'] == "dlTReassembly"

df_dl_results = df_dl_results.sort_values(by=['_time'])
config_df=pd.DataFrame()
config_df=pd.DataFrame()
total_configs_df = pd.DataFrame()
delay_df=pd.DataFrame()
jitter_df=pd.DataFrame()

delay_df['delay'] = list(df_delay_result["_value"])
delay_df['time'] = df_delay_result["_time"]

jitter_df['jitter'] = list(df_jitter_result["_value"])
jitter_df['time'] = df_jitter_result["_time"]

config_df['dlMaxRetxThreshold'] = list(df_dl_results[mask_dlMaxRetxThreshold]["_value"])
config_df['dlPollByte'] = list(df_dl_results[mask_dlPollByte]["_value"])
config_df['dlPollPDU'] = list(df_dl_results[mask_dlPollPDU]["_value"])
config_df['dlTPollRetr'] = list(df_dl_results[mask_dlTPollRetr]["_value"])
config_df['dlTProhib'] = list(df_dl_results[mask_dlTProhib]["_value"])
config_df['dlTReassembly'] = list(df_dl_results[mask_dlTReassembly]["_value"])
config_df['time'] = list(df_dl_results[mask_dlTReassembly]["_time"])

merged_inner = pd.merge(left=delay_df, right=jitter_df, left_on='time', right_on='time')
config_df_f = config_df.drop_duplicates(subset=config_df.columns.difference(['time']), keep="first")


as_df=pd.DataFrame()
range_df = pd.DataFrame(columns = ["start_time", "end_time",
                                   "min_delay", "max_delay", "mean_delay", "median_delay",
                                   "min_jitter", "max_jitter" , "mean_jitter", "median_jitter",])


as_df = merged_inner
columns=['dlMaxRetxThreshold', 'dlPollByte', 'dlPollPDU', 'dlTPollRetr','dlTProhib','dlTReassembly']
calc_columns=['min_delay', 'max_delay', 'mean_delay', 'median_delay','min_jitter', 'max_jitter', 'mean_jitter', 'median_jitter']

config_df_f = config_df_f.sort_values(by=['time'])

for index, row in config_df_f.iterrows():
    addConfig(row)

# Definimos el tiempo de inicio y fin
start_time = as_df['time'].iloc[0]
end_time = as_df['time'].iloc[len(as_df)-1]

automaton=pd.DataFrame()
# Creamos una lista con los tiempos incrementales
times = pd.date_range(start_time, end_time, freq='10s')
# Creamos el DataFrame con la columna de tiempo
automaton = pd.DataFrame({'time': times})


for index, row in automaton.iterrows():
    if (index < (len(automaton)-1)):
            time_for = automaton.iloc[index]['time']
            time_for_next = automaton.iloc[index+1]['time']
            df_sub = as_df[as_df['time'].between(time_for, time_for_next)]
            if (df_sub.empty == False):
                new_row = {"start_time":time_for,
                           "end_time":time_for_next,
                           'min_delay':df_sub['delay'].min(),
                           'max_delay':df_sub['delay'].max(),
                           'mean_delay':df_sub['delay'].mean(),
                           'median_delay':df_sub['delay'].median(),
                           'min_jitter':df_sub['jitter'].min(),
                           'max_jitter':df_sub['jitter'].max(),
                           'mean_jitter':df_sub['jitter'].mean(),
                           'median_jitter':df_sub['jitter'].median(),
                           }
                range_df = range_df.append(new_row, ignore_index=True)
                for column in calc_columns:
                    automaton.at[index,column] = new_row[column]

            elif (index == len(automaton)-1):
                time_for = automaton.iloc[index]['time']
                df_sub = as_df[as_df['time'] > time_for]
                if (df_sub.empty == False):
                    new_row = {"start_time":time_for,
                            "end_time":'',
                            'min_delay':df_sub['delay'].min(),
                            'max_delay':df_sub['delay'].max(),
                            'mean_delay':df_sub['delay'].mean(),
                            'median_delay':df_sub['delay'].median(),
                            'min_jitter':df_sub['jitter'].min(),
                            'max_jitter':df_sub['jitter'].max(),
                            'mean_jitter':df_sub['jitter'].mean(),
                            'median_jitter':df_sub['jitter'].median(),
                            }
                    for column in calc_columns:
                        automaton.at[index,column] = new_row[column]
    

for index, row in automaton.iterrows():
    configAdd = getConfig(config_df.sort_values(by=['time']), row['time'])
    for column in columns:
        automaton.at[index,column] = configAdd[column]

automaton['profile_id'] = '1'
automaton['config_id'] = ''

for index, row in automaton.iterrows():
    automaton.at[index,'config_id'] = getConfigID(row)

automaton_proc = automaton.groupby('config_id').agg({'min_delay': ['min'], 'max_delay': ['max'], 'min_jitter': ['min'], 'max_jitter': ['max']})

automaton_proc['config_id'] = automaton_proc.index

In [ ]:
def addNetworkState(min_delay, max_delay, min_jitter, max_jitter, profile_id, config_id, repetitions):
    global automaton_network_states
    new_row = {'min_delay':min_delay,
               'max_delay':max_delay,
               'min_jitter':min_jitter,
               'max_jitter':max_jitter,
               'profile_id':profile_id,
               'config_id':config_id,
               'repetitions':repetitions}
    automaton_network_states = automaton_network_states.append(new_row, ignore_index=True)

def updateNetworkState(min_delay, max_delay, min_jitter, max_jitter, profile_id, config_id, repetitions):
    global automaton_network_states
    for i in range(len(automaton_network_states)):
        if (automaton_network_states.iloc[i]['min_delay'] == min_delay and
            automaton_network_states.iloc[i]['max_delay'] == max_delay and
            automaton_network_states.iloc[i]['min_jitter'] == min_jitter and
            automaton_network_states.iloc[i]['max_jitter'] == max_jitter and
            automaton_network_states.iloc[i]['profile_id'] == profile_id and
            automaton_network_states.iloc[i]['config_id'] == config_id):
            automaton_network_states.at[i,'repetitions'] = automaton_network_states.iloc[i]['repetitions'] + repetitions
            return
    addNetworkState(min_delay, max_delay, min_jitter, max_jitter, profile_id, config_id, repetitions)


def getNetworkStateID(qty, profile_id, config_id):
    global automaton_network_states
    for i in range(len(automaton_network_states)):
        if (automaton_network_states.iloc[i]['qty'] == qty and
            automaton_network_states.iloc[i]['profile_id'] == profile_id and
            automaton_network_states.iloc[i]['config_id'] == config_id):
            return i
    return -1

def getqtyID(row):
    global qty_df
    for i in range(len(qty_df)):
        if (qty_df.iloc[i]['min_delay'] <= row['max_delay']['max'] and
            qty_df.iloc[i]['max_delay'] >= row['max_delay']['max'] and
            qty_df.iloc[i]['min_jitter'] <= row['max_jitter']['max'] and
            qty_df.iloc[i]['max_jitter'] >= row['max_jitter']['max']):
            return qty_df.iloc[i]['qty']
    return 0


#qty definition
data = {'qty': [-18, -15, -5, 1, 2, 3, 4], 'min_delay': [10000000, 0, 10000000, 0, 6000000, 0, 6000000], 'max_delay': [2000000000000, 10000000, 2000000000000, 6000000, 10000000, 6000000, 10000000], 'min_jitter': [0, 10000000, 10000000, 0, 0, 1000000, 1000000], 'max_jitter': [10000000, 2000000000000, 2000000000000, 1000000, 1000000, 10000000, 10000000]}
qty_df = pd.DataFrame(data)


for index, row in automaton_proc.iterrows():
    automaton_proc.at[index,'qty'] = getqtyID(row)


final_automaton = automaton_proc[['config_id','qty']].reset_index(drop=True)

#print(final_automaton)
final_automaton


In [ ]:
G = nx.from_pandas_edgelist(final_automaton, 'config_id', 'qty')

from matplotlib.pyplot import figure
figure(figsize=(10, 8))
nx.draw_shell(G, with_labels=True)